https://www.analyticsvidhya.com/blog/2021/12/text-classification-of-news-articles/

ปัญหาของข้อมูลชุดนี้


In [116]:
import pandas as pd
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

In [117]:
path = '../wordcloud_style/'
font = path+'Chomsky.otf'
pics = path+'pics.png'

#### import data from cleansing

In [118]:
df = pd.read_csv('../data/cleaned/cleaned-data.csv')

In [119]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4480 entries, 0 to 4479
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   date      4480 non-null   object
 1   label     4480 non-null   object
 2   headline  4480 non-null   object
 3   snippet   4472 non-null   object
 4   url       4480 non-null   object
dtypes: object(5)
memory usage: 175.1+ KB


#### Values Checking

In [120]:
def find_string(a, b, c):
    '''
    ฟังชั่นสำหรับหา String จาก column อย่างเฉพาะเจาะจง
    a = DataFrame Column
    b = 
    c = ตัวอักษรที่จะค้นหา
    Return = DataFrame
    '''
    string = []
    index = []
    for i in df[a]:
        string.append(i.find(c))
    for j in df.index:
        index.append(j)
    df[b] = string
    df['index'] = index
    return df[['index',a,b]].loc[df[b] > -1]

find_string('url', 'ind', 'politics')

,index,url,ind
9,9,https://www.nytimes.com/2019/12/31/us/politics...,38
33,33,https://www.nytimes.com/2020/01/01/us/politics...,38
38,38,https://www.nytimes.com/2020/01/01/us/politics...,38
68,68,https://www.nytimes.com/2020/01/01/us/politics...,38
85,85,https://www.nytimes.com/2020/01/01/us/politics...,38
...,...,...,...
4461,4461,https://www.nytimes.com/2020/01/31/us/politics...,38
4467,4467,https://www.nytimes.com/2020/01/31/us/politics...,38
4469,4469,https://www.nytimes.com/2020/01/31/us/politics...,38
4474,4474,https://www.nytimes.com/2020/01/31/us/politics...,38


In [121]:
def count_values(a):
    '''
    a = ชื่อ column
    ฟั่งชั่นนับจำนวน Label ทั้งหมด
    Return = DataFrame 
    '''
    label = []
    u_names = []
    count = []
    for i in df[a]:
        label.append(i)
    for names in label:
        if names not in u_names:
            names = str(names)
            u_names.append(names)
    for num in u_names:
        count.append(label.count(num))
    return pd.DataFrame({'Label': u_names, 'Numbers': count})

Num_values = count_values('label')
Num_values[['Label', 'Numbers']].loc[Num_values['Numbers'] > 200]

,Label,Numbers
0,us,287
1,opinion,503
5,world,482
6,politics,494
8,arts,346
9,business,297
18,sports,226


In [122]:
df

,date,label,headline,snippet,url,ind,index
0,2020-01-01,us,‘Battling a Demon’: Drifter Sought Help Before...,The gunman who shot two parishioners at the We...,https://www.nytimes.com/2019/12/31/us/texas-ch...,-1,0
1,2020-01-01,opinion,Protect Veterans From Fraud,Congress could do much more to protect America...,https://www.nytimes.com/2019/12/31/opinion/for...,-1,1
2,2020-01-01,health,F.D.A. Plans to Ban Most E-Cigarette Flavors b...,The tobacco and vaping industries and conserva...,https://www.nytimes.com/2019/12/31/health/e-ci...,-1,2
3,2020-01-01,crosswords,‘It’s Green and Slimy’,Christina Iverson and Jeff Chen ring in the Ne...,https://www.nytimes.com/2019/12/31/crosswords/...,-1,3
4,2020-01-01,us,"Corrections: Jan. 1, 2020",Corrections that appeared in print on Wednesda...,https://www.nytimes.com/2019/12/31/pageoneplus...,-1,4
...,...,...,...,...,...,...,...
4475,2020-01-31,sports,Lakers Fall to Blazers on Emotional Night Hono...,It was the Lakers’ first game since Bryant and...,https://www.nytimes.com/2020/01/31/sports/bask...,-1,4475
4476,2020-01-31,sports,Alberto Salazar Is Suspended by SafeSport Afte...,The famed running coach was already barred fro...,https://www.nytimes.com/2020/01/31/sports/olym...,-1,4476
4477,2020-01-31,health,"CPR, by Default","When very old patients suffer cardiac arrest, ...",https://www.nytimes.com/2020/01/31/health/cpr-...,-1,4477
4478,2020-01-31,politics,Impeachment Trial Highlights: A Showdown Over ...,Senators rejected a call for additional witnes...,https://www.nytimes.com/video/us/politics/1000...,33,4478


#### Word Cloud

In [123]:
image = Image.open(pics)

In [124]:
mask = np.array(image)

In [130]:
def word_cloud(a,b,c):
    string = ''
    for i in df[a].loc[df[b] == c]:
        string +=  str(i)
    wordcloud = WordCloud(
                     font_path=font, 
                      relative_scaling = 0.3,
                      min_font_size=1,
                      background_color="white",
                      width=10000,
                      height=1000,
                      max_words=2000, 
                      mask=mask,
                      colormap='plasma', 
                      scale=3,
                      font_step=4,
                    #   contour_width=3, 
                    #   contour_color='steelblue', 
                      collocations=False,
                    
                      margin=2).generate(string)
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")
    plt.show()    
    return plt.savefig(c + '.png')

In [ ]:
word_cloud('snippet', 'label', 'arts')